# STAT 628 Module 2 Report

_Yanqi Huang_, _Huiyu Jiang_, _Xinjie Ye_

## Introduction

**_Yelp_** is an American corporation headquartered in San Francisco. The **_Yelp.com_** and the **_Yelp_** mobile app, which pubish customers' reviews about local businesses and provide online reservation service, are owned by the corporation. _Yelp_ dataset is a part of the **_Yelp Dataset Challenge_**. The project explores the relationship between text, rate and other attributes of a review based on the dataset.

## Background of Yelp Dataset

The Yelp dataset used in this project is derived from **_Yelp Dataset Challenge_**, which encourages students to conduct research and analysis on its dataset. The challenge usually contains photo classification, natural language processing & sentiments a analysis and graph mining. 

The dataset consisting of 9 attributes has about 1.5 million records. The 8 attributes are stars of the review, name of the business, review text, date of the review, city in which the business is located, longitude , latitude and business categories. 

Our project aims at exploring the sentiments contained by reviews and developing a reasonable and practical model to predict stars given by customers.

## Data Exploration

### Text Process

**Generating Sentimental Words and Symbols List:** Our method is using some meaningful words and punctuations to grade each review. We aim at generating a sentimental words list initially. Because the original dataset is too large, we ramdomly sample 100,000 reviews as the source. The first step is tansfering all "n't" to "not". Punctuations are also deleted and capital letters are converted to lower-case letters. Then calculate all words' frequency accroding to review stars. Below is some typical samples.

A | B 
- | - 
![Negative](images/float.png)|![Positive](images/heaven.png)
![Negative](images/egg.png)|![Positive](images/trust.png)

According to the plots above, we know that the key word usually have a clear tendency in distribution. Negative words appear in low rating reviews more, while positive words have a high frequency in 4 and 5 stars reviews. And words have a distribution, which has a shape similar to bell and the peak around 3 stars are defined as neutral words. What's more, the words having concave distributions usually express extreme emotions because they apear in 1 and 5 stars reviews more frequently. It's easy to find that these meaningful words have a common feature, their frequencies of each star vary a lot.

The way to select this kind of useful word is to calculate total frequencies and standard error of frequencies:  

\begin{equation*}
F=\frac{The \ total \ count \ of \ a \ word's \ appearance}{The \ total \ count \ of \ reviews}
\end{equation*}

\begin{equation*}
SD=\sqrt{\frac{\sum_{i=1}^5(F_i-\bar{F})^2}{4}}
\end{equation*}

When the frequency is larger than 0.01 and SD is larger than 0.15, we select the word as a useful word. In terms of special symbols like ":)" and ":(", we also apply this thoery.

**Data Tranformation:** After generating the list, we transform the original review text to a matrix with column names of these useful words and values of useful words counts in each review. We will use these features to construct our model.

## Model Fitting

### Overview

Our cleaned data is a 15*2339 sparse matrix, where each row represents a review and each column represents a feature. Generally speaking, our data has three types of feature:
* **word feature:** Words from useful word list generated by text process
* **statistics:** wordcount, UpperCount, etc
* **others:** special expression, name, date, categories, etc

Then we apply two kinds of eXtreme Gradient Boosting(xgboost), linear regression and multilevel classification, to train models. Following we call the linear model as ** reg:linear ** and classification model as **multi:softmax**.

### Performance

There we both set learning rate of each step as 0.1 and below are the two plots of model errors:

![Negative](images/comp1.png)

Three findings based on the plots:
* **Generally speaking**, two model both have better results when number of rounds increases and the gradient both decrease.

* **reg:linear** model's RMSE decreases very rapidly in its first 50 rounds. After that, the RMSE seems flat but still have little improvement, which from 0.85 to 0.73.

* **multi:softmax** model's Merror starts at a better point than reg:linear and also has a rather rapid decrease rate. However, after ~50 rounds, they all reach the similar level of error rate. Also, with number of round increasing, the training Merror keep decreasing, which may reflects potential overfitting, because test Merror seems flat there.

Also, we plot top 10 features according to their importance in both model:

![Negative](images/comp2.png)

Some findings based on the plots:

* **word feature:** "not", "worst", "delicious", "amazing", "great" are common words in both model. As we can see, the first three may lead to negative reviews while others are positive ones. Others like "no", "horrible" and "terrible" appear in reg:linear model while "but", "best" and "good" appear in multi:softmax model.
* **statistics:** wordCount appears to be very useful in both models. As we further explore this feature, it plays a role as intensive word.
* **others:** exclaim(!) appears to be very useful in both models. From previous analysis, it plays as a intensive word.


### Improvement

As two models both perform rather unsatisfying, which reg:linear has RMSE of 0.73 and multi:softmax has RMSE of 0.75 (from kaggle feedback), we consider to do model merging to improve the rate. Here we apply weighted mean on each predictions with weight assgined as their performance. In other words, the weight of reg:linear is 0.73/(0.73+0.75) while the weight of multi:softmax is 0.75/(0.73+0.75). As a result, final RMSE rate of merged model is 0.709.

## Strength and weakness

### Strength

For model fitting and prediction, more variables are better, while some of them are not so important. We can make sure our useful words are all typical sentiment words.

### Weakness

* Due to time limit, we only use xgboost to construct model. All our exploration in model is adjusting the parameters of xgboost. Due to the inner restriction of our processed data, the improvement got vanished gradually. If we add more variables, the effect of xgboost may be better.

* We set a strict standard to select useful words, resulting in a lack of meaningful variables. After all our attempts to improve the accuracy of test, we realized that the bad outcomes can be caused by interior drawbacks in processed data. We select words of frequencies larger than 0.01, while that means these words appear in all reviews for more than 15,000 times. Actually the number is very large, we can lower it.

## Conclusion

We use the feature engineering to clean the raw data and generate a sparse matrix, and construct our model by xgboost. The **train set RMSE **is about 0.36, while the **test set RMSE ** on Kaggle is about 0.708, which means that our model tends to overfit the data.

## Contribution

| Name              | Contribution               | 
| :----------------| :-----------------------------: | 
|Huiyu Jiang| dealt with feature engineering, data clean and made ppt slides|
| Yanqi Huang| made shiny app, cleaned data and wrote half of notebook report|
|Xinjie Ye| wrote half of notebook report and trained xgboost model|